In [8]:
##  Eval XiJin's First model

import cv2
import numpy as np
from PIL import Image
import sys
sys.path.insert(0,'./caffe/python')
import caffe
import os

import scipy.io as sio    
import matplotlib.pyplot as plt  
plt.rcParams['figure.figsize']=[15,15]
import time
import scipy.misc

INPUT_CHANNEL = 3
MEAN = [114.578, 115.294, 108.353]
DATASET = 'douyu_7100'
n_cl = 2

Mean = np.array((114.578, 115.294, 108.353), dtype=np.float32)
Std = 0.015625

# load net
DEPLOY = '/home/dalong/Workspace/human_matting/fcn_seg/xinjin_model/VGG16_1c_dep.prototxt'
CAFFEMODEL = '/home/dalong/Workspace/human_matting/fcn_seg/xinjin_model/vgg_co_1labels_minAug_3c_iter_11000.caffemodel'
net = caffe.Net(DEPLOY, CAFFEMODEL, caffe.TEST)
caffe.set_mode_gpu();
caffe.set_device(0);

imglist = open('./data/'+DATASET+'/test.txt').readlines()
datanum = len(imglist)

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n,n)

hist = 0

for imgname in imglist:
    imgname = imgname.strip()
    
    # load image, switch to BGR, subtract mean, and make dims C x H x W for Caffe
    img = cv2.imread('./data/'+DATASET+'/Images/'+imgname+'.png')
    img = cv2.resize(img,(192,320))
    img = np.float32(img)
    img -= Mean
    img *= Std
    img = img.transpose((2,0,1))
    
    # load label
    label = sio.loadmat('./data/'+DATASET+'/Labels/'+imgname+'.mat')['label']
    label = np.array(label, dtype=np.float32)
    
    if INPUT_CHANNEL==4:        
        # load 4-th channel
        cimg = cv2.imread('./data/'+DATASET+'/pred_xijin/{}.png_pred.png'.format(imgname),cv2.IMREAD_GRAYSCALE)
        if cimg is None:
            print 'none cimg!!!', imgname
            continue
        cm = np.ndarray((1,600,376),dtype=np.float32)
        cimg = np.float32(cimg)
        cimg = (cimg-128)*Std
        cm[0,:,:] = cimg
        
        
    net.blobs['data'].reshape(1,*img.shape)
    net.blobs['data'].data[...] = img

    # run net and take argmax for prediction
    net.forward()
    out = net.blobs['mask/prob'].data[0]
    out = out.transpose().reshape((32,18,2))[:,:,1]
    out = cv2.resize(out*255,(368,640))
    out[np.where(out<255*0.3)] = 0
    out[np.where(out>=255*0.3)] = 1
    out = np.array(out, dtype=np.int64)

    this_hist =  fast_hist(label.flatten(),out.flatten(),n_cl)
    this_hist = np.array(this_hist, dtype=np.float32)
    this_iu = np.diag(this_hist) / (this_hist.sum(1) + this_hist.sum(0) - np.diag(this_hist))
    if np.isnan(this_iu[0]):
        this_iu[0] = 1
    if np.isnan(this_iu[1]):
        this_iu[1] = 1
    print imgname, this_iu, np.mean(this_iu) # first is neg iou, second is pos iou 
    hist += this_hist
    
hist = np.array(hist, dtype=np.float32)
iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
print '================================= result ================================='
print 'INPUT_CHANNEL: ',INPUT_CHANNEL
print 'MEAN', MEAN
print 'DEPLOY: ',DEPLOY
print 'CAFFEMODEL: ',CAFFEMODEL
print 'TEST DATASET: ',DATASET,
print 'TEST DATASET NUM: ',datanum
print 'IOU: ', iu, np.mean(iu)    






660209_1489763036_0 [ 0.35302579  0.93883193] 0.645929
douyuTV_660209_1488641433_5 [ 0.64653093  0.93923938] 0.792885
douyuTV_769170_1488942330_5 [ 0.73128492  0.96114147] 0.846213
douyuTV_1551804_1488811840_15 [ 0.92203462  0.95814341] 0.940089
douyuTV_579386_1488532523_0 [ 0.58850276  0.92371666] 0.75611
douyuTV_586697_1488550043_15 [ 0.85231823  0.96057022] 0.906444
douyuTV_825993_1489069177_0 [ 0.92370772  0.6309095 ] 0.777309
douyuTV_617410_1488552926_0 [ 0.86615717  0.95257884] 0.909368
douyuTV_1255190_1488457622_15 [ 0.91447914  0.976255  ] 0.945367
douyuTV_586697_1489049502_0 [ 0.91925031  0.96543503] 0.942343
786909_1489894778_0 [ 0.66832113  0.95948952] 0.813905
438573_1489888999_0 [ 0.7638092   0.90907109] 0.83644
douyuTV_1551804_1488458252_10 [ 0.91326958  0.97157109] 0.94242
976326_1489783715_0 [ 0.85809684  0.95774853] 0.907923
684244_1489851864_0 [ 0.77986938  0.88009   ] 0.82998
1255190_1489847960_0 [ 0.55697399  0.83598   ] 0.696477
douyuTV_1546683_1488875586_0 [ 0.906

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:82: RuntimeWarning: invalid value encountered in divide


1546683_1489693407_0 [ 0.87519401  0.88770831] 0.881451
douyuTV_1773564_1488861592_0 [ 0.3935197   0.91381639] 0.653668
douyuTV_1642432_1489058512_0 [ 0.94050974  0.9475956 ] 0.944053
douyuTV_459790_1488952692_0 [ 0.83184963  0.84361696] 0.837733
976326_1489696252_0 [ 0.01344156  0.93543494] 0.474438
438573_1489633381_0 [ 0.77463746  0.97344559] 0.874042
1255190_1489774789_0 [ 0.87732184  0.97959828] 0.92846
douyuTV_660209_1488903077_0 [ 0.84671533  0.97329521] 0.910005
douyuTV_1242473_1488895441_0 [ 0.58401299  0.89486867] 0.739441
douyuTV_604944_1488811240_10 [ 0.86706227  0.95847785] 0.91277
1149200-0 [ 0.35864478  0.95526433] 0.656955
1546683_1488188720-0 [ 0.77134138  0.80218607] 0.786764
douyuTV_1192758_1488808599_0 [ 0.79156941  0.807679  ] 0.799624
douyuTV_182102_1489044245_0 [ 0.72153705  0.97531646] 0.848427
douyuTV_1794142_1488501131_10 [ 0.79265004  0.94644696] 0.869548
douyuTV_825993_1488633957_0 [ 0.90923321  0.95803028] 0.933632
douyuTV_1730418_1488941879_0 [ 0.688039   